<a href="https://colab.research.google.com/github/joomj2000/Chatbot/blob/main/6_vectorstore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain langchain-community langchain-core
!pip install tiktoken sentence_transformers pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 941.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.9/379.9 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.2/140.2 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 4.7 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata 

In [3]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/[정책브리프 2021-04] 탄소중립 대응을 위한 정부 정책과 동향.pdf")
pages = loader.load_and_split()

In [4]:
import tiktoken

tokenizer = tiktoken.get_encoding('cl100k_base')

def tiktoken_len(text):
  tokens = tokenizer.encode(text)
  return len(tokens)

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=tiktoken_len,
)

docs = text_splitter.split_documents(pages)

In [6]:
from langchain.embeddings import HuggingFaceEmbeddings

model_name = 'jhgan/ko-sbert-nli'
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
ko = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/sett

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.46k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/538 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
!pip install ChromaDB

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 4.8 MB/s eta 0:00:00
  

In [8]:
from langchain.vectorstores import Chroma

db=Chroma.from_documents(docs, ko)

In [9]:
query= '장기저탄소발전전략'

docs = db.similarity_search(query)

[Document(metadata={'page': 7, 'source': '/content/[정책브리프 2021-04] 탄소중립 대응을 위한 정부 정책과 동향.pdf'}, page_content='05이노비즈 정책브리프\n장기저탄소발전전략 (LEDS)\n∙우리나라는 녹색성장위원회 심의와 국무회의 의결을 거쳐 지난해 말 LEDS 를 UN에 제출하였으며 \n부문별 전략은 다음과 같음\n- (에너지공급 ) 화석연료 발전 중심의 전력공급 체계를 재생에너지와 그린수소 중심으로 전환하고 이산화탄\n소포집 기술 등을 적극적으로 활용하여 전력부문의 탄소중립을 달성\n- (산업) 미래 신기술 , 에너지효율 향상, 순환경제 실현으로 지속가능한 탄소중립 산업 생태계를 구축\n- (수송) 청정 에너지원 (전기·수소)을 동력으로 하는 수송수단 (자동차 , 철도, 항공기 , 선박)을 확대하고 디지\n털 기술을 활용한 자율주행차 , 교통 수요관리를 통해 수송부문 탄소중립 기반을 조성\n- (건물) 단열과 기밀성능을 강화하고 에너지고효율 제품 사용을 확대하여 건물에서 사용되는 에너지를 최'),
 Document(metadata={'page': 6, 'source': '/content/[정책브리프 2021-04] 탄소중립 대응을 위한 정부 정책과 동향.pdf'}, page_content='보제공 확대 \n- 재생원료 사용 극대화 , 철강, 플라스틱을 대체하는 혁신소재 개발, 탄소발자국 * 등 친환경 제품 정보 제\n공 확대 등 순환 사회로의 전환 가속화\n*탄소발자국이란 개인 또는 기업, 국가 등의 활동이나 상품을 생산하고 소비하는 전체 과정을 통해 발생시키는 온실가스 , 특히 이산화탄소의 \n총량을 의미함\n ③ 탄소중립 사회로의 공정전환\n- (취약 산업·계층 보호) 내연기관차 완성차 및 부품업체 등 축소산업에 대한 R&D, M&A 등을 통해 대\n체·유망분야로 사업전환 적극 지원, 맞춤형 재취업 지원\n- 구조전환으로 축소되는 석탄발전 ‧내연기관차 산업 등의 경우, 대체‧유망분야로의 

In [10]:
db2=Chroma.from_documents(docs, ko,persist_directory='./chroma_db')

In [11]:
db3=Chroma(persist_directory='./chroma_db',embedding_function=ko)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(


In [12]:
docs=db3.similarity_search(query)
print(docs[0].page_content)

05이노비즈 정책브리프
장기저탄소발전전략 (LEDS)
∙우리나라는 녹색성장위원회 심의와 국무회의 의결을 거쳐 지난해 말 LEDS 를 UN에 제출하였으며 
부문별 전략은 다음과 같음
- (에너지공급 ) 화석연료 발전 중심의 전력공급 체계를 재생에너지와 그린수소 중심으로 전환하고 이산화탄
소포집 기술 등을 적극적으로 활용하여 전력부문의 탄소중립을 달성
- (산업) 미래 신기술 , 에너지효율 향상, 순환경제 실현으로 지속가능한 탄소중립 산업 생태계를 구축
- (수송) 청정 에너지원 (전기·수소)을 동력으로 하는 수송수단 (자동차 , 철도, 항공기 , 선박)을 확대하고 디지
털 기술을 활용한 자율주행차 , 교통 수요관리를 통해 수송부문 탄소중립 기반을 조성
- (건물) 단열과 기밀성능을 강화하고 에너지고효율 제품 사용을 확대하여 건물에서 사용되는 에너지를 최


In [13]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 31.2 MB/s eta 0:00:00


In [14]:
from langchain.vectorstores import FAISS

db_faiss = FAISS.from_documents(docs, ko)

In [16]:
query='삼성전자의 동향'
result=db_faiss.similarity_search(query)
print(result[0].page_content)

속화 추진
- 산업 분야에서는 재생에너지 , 수소, 에너지 IT 등 3대 에너지 신산업 육성에 집중
- (고탄소 산업구조 혁신) 철강, 석유화학 등 탄소 다배출 업종에 대한 기술개발 지원, 고탄소 중소기업 대
상 맞춤형 공정개선 지원 등을 통해 저탄소 산업구조로의 전환 추진
- 금속･화학제품제조업 등 다배출 중소기업을 대상으로 공정진단을 통해 맞춤형 공정개선 ‧설비보급을 지원
하는 등 산업 전과정 (연‧원료→공정→제품→소비‧자원 순환)에서의 저탄소 전환을 촉진
- (미래모빌리티로 전환) 친환경차 가격·충전·수요 혁신을 통해 수소·전기차 생산, 보급 확대, 전국 2천만 
세대 전기차 충전기 보급, 도시·거점별 수소 충전소 구축
- (도시·국토 저탄소화 ) 신규 건축물 제로에너지 건축 의무화 , 국토 계획 수립 시 생태자원 활용한 탄소흡
수 기능 강화 
 ② 신유망 저탄소산업 생태계 조성


In [17]:
result_scores=db.similarity_search_with_score(query)
print(result_scores[0])

(Document(metadata={'page': 9, 'source': '/content/[정책브리프 2021-04] 탄소중립 대응을 위한 정부 정책과 동향.pdf'}, page_content='07이노비즈 정책브리프\n04 국내 대기업 동향 3)\n∙SK그룹\n- ‘20년 11월 국내 최초로 RE100 프로젝트에 가입\n- 2025년까지 이천·청주 사업장의 1톤 차량을 전기 자동차로 교체할 예정\n- SK하이닉스는 공정가스 제거시스템을 적용하는 사업장을 확대함\n∙ 삼성전자\n- 2019년 온실가스 배출량을 ‘18년 대비 8.9%, ‘08년 대비 70% 감축\n- 수원사업장과 기흥사업장에 각각 1.9매가와트 , 1.5메가와트 규모의 태양광발전 설비 설치\n- 제품 사용단계에서의 고효율 제품 및 저전력 반도체 개발을 통해 전력 효율화 추진\n∙ LG그룹\n- 15년째 세계 최대 지구촌 전등 끄기 캠페인인 ‘어스아워 ’에 동참\n- 이사회 산하에 ESG 위원회 신설\n- LG전자는 2030년까지 탄소중립을 달성하겠다는 목표를 수립하고 , 포스코와 협력하여 해수열냉난방시스\n템, 건물일체형태양광 등 신재생에너지 관련 기술 개발에 적극 투자\n∙ 현대제철'), 0.961801290512085)


In [18]:
query = '탄소중립사회'
results = db.similarity_search_with_relevance_scores(query, k=3)

print('질문:' + query)

for i in range(len(results)):
  print('{0}번째 유사도 문서 \n{1}'.format(i+1, round(results[i][1],2)))
  print('-' * 100)
  print(results[i][0].page_content)
  print('-' * 100)
  print(results[i][0].metadata)

질문:탄소중립사회
1번째 유사도 문서 
0.66
----------------------------------------------------------------------------------------------------
01이노비즈 정책브리프
01 탄소중립과 대응 필요성
탄소중립이란 ?
∙탄소중립은 개인·회사·단체 등에서 배출되는 탄소와 흡수되는 탄소량을 같게 해 탄소 ‘순배출이 제
로(0)’가 되게 하는 것으로 , ‘넷-제로(Net-Zero)’ 라 부르기도 함
- 인간의 활동에 의한 온실가스 * 배출을 최대한 줄이고 , 남은 온실가스는 흡수, 제거하여 ‘순배출이 제로
(0)’가 되는 개념이며 , 기후변화에 대응하기 위한 글로벌 추세로 자리 잡고 있음
*온실가스란 ‘지구 대기를 오염시켜 온실 효과를 일으키는 가스’를 통틀어 이르는 말로, 이산화탄소 (CO2), 메탄(CH4), 아산화질소 (N2O), 수
소불화탄소 (HFCs), 과불화탄소 (PFCs), 육불화황 (SF6) 등은 배출 규제 및 거래의 대상이 되고 있음 
대응 필요성
∙지구 온난화로 인한 폭염, 폭설, 태풍, 산불 등 이상기후 현상이 세계 곳곳에서 발생되고 있는 가
운데 기후변화 문제에 대한 국제사회의 공동 대응 필요성이 대두됨
----------------------------------------------------------------------------------------------------
{'page': 3, 'source': '/content/[정책브리프 2021-04] 탄소중립 대응을 위한 정부 정책과 동향.pdf'}
2번째 유사도 문서 
0.64
----------------------------------------------------------------------------------------------------
보제공 확대 
- 재생원료 사용 극대화 , 철강, 플라스틱을 대체하는 혁신소재 개발, 탄소발자국 * 등 친환경 제품 정보 제
공 확대 등 순환 사